In [3]:
%reload_ext Cython

In [16]:
%%cython -f
# cython: profile=True
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

cimport cython
from cython.parallel import parallel, prange
from libc.stdlib cimport rand, RAND_MAX

import time
import cupy as cp

from random import random
from cpython cimport array
import numpy as np
cimport numpy as np

ctypedef np.int_t DTYPE_t
cdef bint DEBUG


## DATA GENERATION
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef generateData(int i, int j):
    
    cdef long startTime = time.time() 
    
    cdef double[:,:] output = np.zeros((i, j))
    cdef int i_n, j_n
    
    with nogil, parallel():
        for i_n in prange(i):
            for j_n in prange(j):
                output[i_n, j_n] = j_n + 1
    
    for row in output:
        np.random.shuffle(row)

    print("Data generation took", 1000*(time.time() - startTime), "ms")
    return cp.array(output, dtype=np.int)

        
cdef class AbstractSMA:
    
    cdef public:
        
        # Timestep (t), size (a, b)
        int t, a, b
        
        # Preferences (PA, PB), Matches (MA, MB), History (HA, HB)
        PA, PB
        MA, MB
        HA, HB
        
    def __init__(self, PA, PB):
        
        self.a, self.b = PA.shape[0], PB.shape[1]
        self.PA, self.PB = PA, PB
        self.MA = self.MB = self.HA = self.HB = cp.zeros((self.a, self.b), dtype=np.int)
        
        if(DEBUG): print("\nPA (*)\n", self.PA, "\n\nPB\n", self.PB, "\n")
    
    @cython.boundscheck(False)
    @cython.wraparound(False)
    cdef AbstractSMA nextState(self):
        
        cdef:
            # proposalMask: where pref is selected rank
            # singleMask: where row is all zeros
            singleMask = cp.all(self.MA == 0, axis = 1, keepdims = True)
#             proposalMask = self.PA >= max(self.a - self.t, 1)
            proposalMask = self.PA >= 1
            feasibleProposalMask = proposalMask * singleMask
            
            # proposerScores: reviewers' ratings of proposers
            # reviewerScores: proposers' ratings of reviewers
            AScores = self.PB * feasibleProposalMask
            BScores = self.PA * feasibleProposalMask
            
        # track past proposals
        self.HA = self.HA | feasibleProposalMask
            
        # start @ t = 0
        if DEBUG:
            print("------------\nt =", self.t, "\n")
            print("Single Mask\n", singleMask.astype(np.int), "\n\n")
            print("Proposal Mask\n", proposalMask.astype(np.int), "\n\n")
            print("Feas. Proposal Mask\n", feasibleProposalMask.astype(np.int), "\n\n")
            print("MA (pre-mutate)\n", self.MA, "\n\n")
            print("MB (pre-mutate)\n", self.MB, "\n\n")
            print("A's scores with B\n", AScores, "\n\n")
            print("B's scores with A\n", BScores, "\n\n")
            print("History A\n", self.HA, "\n\n")
        
        # update a match if proposal score is higher than existing match
#         self.MB = cp.maximum(AScores, self.MB)
        self.MB = cp.maximum(self.PB * singleMask, self.MB)
        
        # eliminate non-max matches from matrix (only keep highest)
        self.MB *= self.MB == cp.amax(self.MB, axis = 1, keepdims = True)
        self.MB *= self.MB == cp.amax(self.MB, axis = 0, keepdims = True)
    
        # A's matches = A preferences masked by mask of B
        self.MA = self.PA * (self.MB != 0)
        
        if DEBUG:
            print("final MA\n", self.MA, "\n\n")
            print("final MB\n", self.MB, "\n\n")

        self.t += 1
        return self
    
    cdef bint everyoneIsContent(self):
#         return cp.sum(self.HA) == self.HA.size
        return self.t >= self.a
    
    cdef public AbstractSMA run(self):
        while not self.everyoneIsContent():
            self.nextState()
        
        return self

cdef:
#     PA_small = cp.array([
#         [2, 1, 3],
#         [1, 2, 3],
#         [1, 3, 2]
#     ], dtype=np.int)
    
#     # change to size a x b
#     PB_small = cp.array([
#         [1, 3, 2],
#         [2, 2, 1],
#         [3, 1, 3]
#     ], dtype=np.int)

    PA_small = cp.array([
        [3, 4, 2, 1],
        [2, 4, 3, 1],
        [3, 1, 2, 4],
        [3, 2, 4, 1]
    ])
    
    PB_small = cp.array([
        [3, 1, 2, 4],
        [3, 4, 2, 1],
        [2, 4, 3, 1],
        [3, 2, 4, 1]
    ])
    
    int N = 4
    PA_big = generateData(N, N)
    PB_big = generateData(N, N)

cpdef test():
    return AbstractSMA(PA_big if BIG else PA_small, PB_big if BIG else PB_small).run()

print("Group A is making all proposals for this test case.")

# DEBUG = True
# BIG = False

DEBUG = True
BIG = True

myTest = test()
print("Done. Finished in", myTest.t, "steps.\n")
print("MA\n", myTest.MA, "\n\n")
print("MB\n", myTest.MB, "\n\n")
print("HA\n", myTest.HA, "\n\n")

Data generation took 495.6510066986084 ms
Data generation took 495.9139823913574 ms
Group A is making all proposals for this test case.

PA (*)
 [[4 2 1 3]
 [4 1 3 2]
 [1 4 3 2]
 [4 2 3 1]] 

PB
 [[2 4 1 3]
 [2 1 3 4]
 [4 1 3 2]
 [4 1 3 2]] 

------------
t = 0 

Single Mask
 [[1]
 [1]
 [1]
 [1]] 


Proposal Mask
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]] 


Feas. Proposal Mask
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]] 


MA (pre-mutate)
 [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]] 


MB (pre-mutate)
 [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]] 


A's scores with B
 [[2 4 1 3]
 [2 1 3 4]
 [4 1 3 2]
 [4 1 3 2]] 


B's scores with A
 [[4 2 1 3]
 [4 1 3 2]
 [1 4 3 2]
 [4 2 3 1]] 


History A
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]] 


final MA
 [[0 2 0 0]
 [0 0 0 2]
 [1 0 0 0]
 [4 0 0 0]] 


final MB
 [[0 4 0 0]
 [0 0 0 4]
 [4 0 0 0]
 [4 0 0 0]] 


------------
t = 1 

Single Mask
 [[0]
 [0]
 [0]
 [0]] 


Proposal Mask
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]] 


Feas. P